In [84]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import tensorflow as tf
import pandas as pd
import numpy as np
import pickle as pickle

In [85]:
df = pd.read_csv("Churn_Modelling.csv")
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB
None


In [86]:
Y = df[["Exited"]]
X = df.drop(columns=['RowNumber','CustomerId','Surname','Exited'],axis=1)
# print(Y.head())
print(X.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CreditScore      10000 non-null  int64  
 1   Geography        10000 non-null  object 
 2   Gender           10000 non-null  object 
 3   Age              10000 non-null  int64  
 4   Tenure           10000 non-null  int64  
 5   Balance          10000 non-null  float64
 6   NumOfProducts    10000 non-null  int64  
 7   HasCrCard        10000 non-null  int64  
 8   IsActiveMember   10000 non-null  int64  
 9   EstimatedSalary  10000 non-null  float64
dtypes: float64(2), int64(6), object(2)
memory usage: 781.4+ KB
None


Now we want to do some preprocessing. We want to transform categorical values to numerical values and we want to scale the values. Note that the columns in this experiment refer to the names of the columns. That's why we use the attribute .columns.to_list()

In [87]:
cat_columns = X.select_dtypes(include=[object]).columns.to_list()
num_columns = X.select_dtypes(include=['float64','int64']).columns.to_list()
print(cat_columns, num_columns)
transform = ColumnTransformer(
    transformers=(
        ('num', StandardScaler(), num_columns),
        ('cat', OneHotEncoder(), cat_columns)
        )
    )

data = transform.fit_transform(X)
with open('scaler.pkl', 'wb') as f:
    pickle.dump(transform, f)
print("data columns:", transform.get_feature_names_out())
x_train, x_test, y_train, y_test = train_test_split(data, Y, test_size=0.2, random_state=42)

['Geography', 'Gender'] ['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary']
data columns: ['num__CreditScore' 'num__Age' 'num__Tenure' 'num__Balance'
 'num__NumOfProducts' 'num__HasCrCard' 'num__IsActiveMember'
 'num__EstimatedSalary' 'cat__Geography_France' 'cat__Geography_Germany'
 'cat__Geography_Spain' 'cat__Gender_Female' 'cat__Gender_Male']


In [88]:
print(type(data))
print(data.dtype)
print(x_train.shape, type(x_train))

<class 'numpy.ndarray'>
float64
(8000, 13) <class 'numpy.ndarray'>


Okay so now we've transformed our data. Now we need to create our ML model. 

In [89]:

model = Sequential([
    Dense(64, activation='relu', input_shape=(x_train.shape[1],), use_bias=True),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')]
)
opt = tf.keras.optimizers.Adam(learning_rate=0.01)
loss=tf.keras.losses.BinaryCrossentropy(from_logits=False)
model.compile(
    optimizer=opt,
    loss=loss,
    metrics=['accuracy']
)


/Users/jeltevenekamp/Documents/3. Building, Deploying, and Optimizing Generative AI with Langchain and Huggingface/2. ANN Project Implementation/tf-env/lib/python3.11/site-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Now we need to create a log directory to capture the history and performance of the model. Afterwards we'll train the model.


In [90]:
import datetime
log_dir = "logs/fit/" + "experiment_1" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [91]:
## Set up Early Stopping
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

history = model.fit(x_train, y_train, validation_data=[x_test, y_test], epochs=50, batch_size=50, verbose=1, callbacks=[tensorboard_callback, early_stopping])

Epoch 1/50
160/160 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - accuracy: 0.8002 - loss: 0.4494 - val_accuracy: 0.8560 - val_loss: 0.3483
Epoch 2/50
160/160 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8612 - loss: 0.3431 - val_accuracy: 0.8580 - val_loss: 0.3485
Epoch 3/50
160/160 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.8566 - loss: 0.3450 - val_accuracy: 0.8630 - val_loss: 0.3395
Epoch 4/50
160/160 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8611 - loss: 0.3431 - val_accuracy: 0.8625 - val_loss: 0.3416
Epoch 5/50
160/160 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8566 - loss: 0.3371 - val_accuracy: 0.8635 - val_loss: 0.3359
Epoch 6/50
160/160 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.8636 - loss: 0.3304 - val_accuracy: 0.8595 - val_loss: 0.3405
Epoch 7/50
160/160 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.8672 - loss: 0.3202 - val_accuracy: 0.8570 - val_loss: 0.3428
Epoch 8/50
160/160 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8602 - loss: 0.3329 - val_accuracy: 0.

In [92]:
model.save("model.h5")


In [96]:
## Load Tensorboard Extension
%load_ext tensorboard
%tensorboard --logdir logs/fit/experiment_120250721-111400
# To view the TensorBoard, run the following command in your terminal:
# tensorboard --logdir logs/fit/experiment_1

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6007 (pid 17352), started 0:00:54 ago. (Use '!kill 17352' to kill it.)